# ДЗ1 Вариант 2
## Познянский Михаил ИУ6-54Б

  1. Рассчитайте средний рейтинг товаров из набора данных.
  
  2. Сопоставьте полученные данные из предыдущего пункта с наименованием товаров.
  
  3. Сформируйте RDD товаров с рейтингом меньшим 3. Выведите топ-10 товаров с наименьшим рейтингом.

  4. Сохраните результат в постоянное хранилище.


In [1]:
!pip install pyspark

import os
import shutil
import json
from pyspark.sql import SparkSession

# Создаем сессию
sess = SparkSession.builder \
    .appName("HW1_Electronics") \
    .master("local[*]") \
    .getOrCreate()

sc = sess.sparkContext

# Загрузка файлов
!wget -q http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
!wget -q http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz

print("Environment ready. Data downloaded.")

Environment ready. Data downloaded.


In [2]:
# Читаем данные
raw_reviews_data = sc.textFile("reviews_Electronics_5.json.gz")
raw_meta_data = sc.textFile("meta_Electronics.json.gz")

# 1. Обработка отзывов:
asin_ratings_pair = raw_reviews_data \
    .map(json.loads) \
    .map(lambda r: (r['asin'], r['overall']))

# 2. Обработка товаров:
asin_titles_pair = raw_meta_data \
    .map(eval) \
    .map(lambda p: (p['asin'], p.get('title', 'N/A')))

print(f"Reviews count: {asin_ratings_pair.count()}")
print(f"Products count: {asin_titles_pair.count()}")

Reviews count: 1689188
Products count: 498196


### Пункт 1

In [3]:
# Используем aggregateByKey для подсчета суммы и количества оценок
ratings_agg = asin_ratings_pair.aggregateByKey(
    (0.0, 0),
    lambda stat, rating: (stat[0] + rating, stat[1] + 1),
    lambda stat1, stat2: (stat1[0] + stat2[0], stat1[1] + stat2[1])
)

# Вычисляем среднее арифметическое
mean_ratings = ratings_agg.mapValues(lambda v: v[0] / v[1])

print("Example average rating:", mean_ratings.take(1))

Example average rating: [('0528881469', 2.4)]


### Пункт 2

In [4]:
# Объединяем рассчитанные рейтинги с именами товаров
all_products_with_names = mean_ratings.join(asin_titles_pair)

print("Joined")

Joined


### Пункт 3

In [5]:
# Теперь из полного списка выбираем только плохие товары с рейтингом < 3.0
poor_quality_rdd = all_products_with_names.filter(lambda x: x[1][0] < 3.0)

# Преобразуем структуру для удобства вывода
clean_result = poor_quality_rdd.map(lambda item: (item[1][0], item[1][1]))

# Берем топ-10 товаров с наименьшим рейтингом
worst_10_list = clean_result.takeOrdered(10, key=lambda x: x[0])

print("\nTOP 10 LOWEST RATED ITEMS")
for i, (score, name) in enumerate(worst_10_list, 1):
    print(f"{i}. [{score:.1f}] {name}")


TOP 10 LOWEST RATED ITEMS
1. [1.0] Agfa ePhoto SMILE 0.2MP Digital Camera
2. [1.0] GE 24746 Futura HDTV Ready Antenna
3. [1.0] ATI TV Wonder 200 PCI Video Card w/PVR Capabilities
4. [1.0] Dynex-DX-AP100 Adapter Mini DVI to Mini-DIN
5. [1.0] Maxtor OneTouch 4 - 1 TB USB 2.0 Desktop External Hard Drive STM310005OTA3E1-RK
6. [1.0] Rapid USB Charger Adapters include: Wall and Car + USB Cable for Samsung Galaxy TAB (P1000)
7. [1.0] RCA DRC8335 DVD Recorder &amp; VCR Combo With Built-In Tuner
8. [1.0] StarTech HDMISPL1HH 1 feet Standard HDMI Cable - 1x HDMI (M) to 2x HDMI (F) (Discontinued by Manufacturer)
9. [1.0] Zeikos 57-in-1 USB 2.0 Flash Memory Card Reader ZE-CR201
10. [1.0] NEEWER&reg; Photographic Barn Door &amp; Honeycomb Grid &amp; Gel Set for Alienbees Alienbee Flash


### Пункт 4

In [6]:
# Сохраняем результат в постоянное хранилище
RESULT_DIR = "bad_products_report"

if os.path.exists(RESULT_DIR):
    shutil.rmtree(RESULT_DIR)

output_rdd = clean_result \
    .sortBy(lambda x: x[0]) \
    .map(lambda x: f"{x[0]:.2f}\t{x[1]}")

output_rdd.saveAsTextFile(RESULT_DIR)

print(f"Data saved to '{RESULT_DIR}'")

Data saved to 'bad_products_report'
